## PCA 的思想

降维的两种方式:

+ 特征选择(feature selection)，通过变量选择来缩减维数。
+ 特征提取（feature extraction）,通过线性或非线性变换（投影）来生成缩减集（复合变量）。

主成分分析（PCA）降维基本思想：

1. 将坐标轴中心移到数据的中心，然后旋转坐标轴，使得数据在C1轴上的上的方差最大，即全部n个数据个体在该方向上的投影最为分散。意味着更多的信息被保留下来。C1成为第一主成分
2. C2第二主成分：找一个C2，使得C2与C1的协方差（相关系数）为0，以免与C1信息重叠，并且使数据在该方向的方差尽量最大
3. 以此类推，找到第三主成分，第四主成分。。。。第p个主成分

## 数学角度解读PCA

### 基

一个二维向量可以对应二维笛卡尔直角坐标系中从原点出发的一个有向线段

![](./img/base_vector.png)

一个向量(3,2)实际是说在x轴投影为3而y轴的投影为2，推而广之向量(x,y)实际上表示线性组合

$$x(1,0)^\mathsf{T}+y(0,1)^\mathsf{T}$$

此处(1,0)和(0,1)叫做二维空间中的一组基

![](./img/base.png)

要准确描述向量，首先要确定一组基，然后给出在基所在的各个直线上的投影值，就可以了。假如$(\frac{1}{\sqrt{2}},\frac{1}{\sqrt{2}})$和$(-\frac{1}{\sqrt{2}},\frac{1}{\sqrt{2}})$为一组基,我们想获得(3,2)在新基上的坐标，即在两个方向上的投影矢量值，那么根据内积的几何意义，我们只要分别计算(3,2)和两个基的内积，不难得到新的坐标为$(\frac{5}{\sqrt{2}},-\frac{1}{\sqrt{2}})$

![](./img/transform.png)

基变换用矩阵表示为

$$\begin{pmatrix}
  1/\sqrt{2}  & 1/\sqrt{2} \\
  -1/\sqrt{2} & 1/\sqrt{2}
\end{pmatrix}
\begin{pmatrix}
  3 \\
  2
\end{pmatrix}
=
\begin{pmatrix}
  5/\sqrt{2} \\
  -1/\sqrt{2}
\end{pmatrix}$$


一般的，如果我们有M个N维向量，想将其变换为由R个N维向量表示的新空间中，那么首先将R个基按行组成矩阵A，然后将向量按列组成矩阵B，那么两矩阵的乘积AB就是变换结果，其中AB的第m列为A中第m列变换后的结果

$$
\begin{pmatrix}
  p_1 \\
  p_2 \\
  \vdots \\
  p_R
\end{pmatrix}
\begin{pmatrix}
  a_1 & a_2 & \cdots & a_M
\end{pmatrix}
=
\begin{pmatrix}
  p_1a_1 & p_1a_2 & \cdots & p_1a_M \\
  p_2a_1 & p_2a_2 & \cdots & p_2a_M \\
  \vdots  & \vdots  & \ddots & \vdots \\
  p_Ra_1 & p_Ra_2 & \cdots & p_Ra_M
\end{pmatrix}
$$

其中$p_i$是一个行向量，表示第$i$个基，$a_j$是一个列向量，表示第$j$个原始数据记录。这里R可以小于N，而R决定了变换后数据的维数。也就是说，我们可以将一N维数据变换到更低维度的空间中去，变换后的维度取决于基的数量。因此这种矩阵相乘的表示也可以表示降维变换。

因此我们给矩阵相乘找到了一种物理解释：两个矩阵相乘的意义是将右边矩阵中的每一列列向量变换到左边矩阵中每一行行向量为基所表示的空间中去。更抽象的说，一个矩阵可以表示一种线性变换


## 主成分

选择不同的基可以对同样一组数据给出不同的表示，而且如果基的数量少于向量本身的维数，则可以达到降维的效果。但是我们还没有回答一个最最关键的问题：如何选择基才是最优的。或者说，如果我们有一组N维向量，现在要将其降到K维（K小于N），那么我们应该如何选择K个基才能最大程度保留原有的信息？

![](./img/variance.png)

如果我们必须使用一维来表示这些数据，又希望尽量保留原始的信息，你要如何选择？

通过上一节对基变换的讨论我们知道，这个问题实际上是要在二维平面中选择一个方向，将所有数据都投影到这个方向所在直线上，用投影值表示原始记录。这是一个实际的二维降到一维的问题。

那么如何选择这个方向（或者说基）才能尽量保留最多的原始信息呢？一种直观的看法是：希望投影后的投影值尽可能分散。

以上图为例，可以看出如果向x轴投影，那么最左边的两个点会重叠在一起，中间的两个点也会重叠在一起，于是本身四个各不相同的二维点投影后只剩下两个不同的值了，这是一种严重的信息丢失，同理，如果向y轴投影最上面的两个点和分布在x轴上的两个点也会重叠。所以看来x和y轴都不是最好的投影选择。我们直观目测，如果向通过第一象限和第三象限的斜线投影，则五个点在投影后还是可以区分的。

这种分散程度，可以用数学上的方差来表述。此处，一个字段的方差可以看做是每个元素与字段均值的差的平方和的均值，即：

$$Var(a)=\frac{1}{m}\sum_{i=1}^m{(a_i-\mu)^2}$$

于是上面的问题被形式化表述为：寻找一个一维基，使得所有数据变换为这个基上的坐标表示后，方差值最大。

对于上面二维降成一维的问题来说，找到那个使得方差最大的方向就可以了。不过对于更高维，还有一个问题需要解决。考虑三维降到二维问题。与之前相同，首先我们希望找到一个方向使得投影后方差最大，这样就完成了第一个方向的选择，继而我们选择第二个投影方向。

如果我们还是单纯只选择方差最大的方向，很明显，这个方向与第一个方向应该是“几乎重合在一起”，显然这样的维度是没有用的，因此，应该有其他约束条件。从直观上说，让两个字段尽可能表示更多的原始信息，我们是不希望它们之间存在（线性）相关性的，因为相关性意味着两个字段不是完全独立，必然存在重复表示的信息。

数学上可以用两个字段的协方差表示其相关性,当协方差为0时，表示两个字段完全独立。为了让协方差为0，我们选择第二个基时只能在与第一个基正交的方向上选择。因此最终选择的两个方向一定是正交的。

至此，我们得到了降维问题的优化目标：将一组N维向量降为K维（K大于0，小于N），其目标是选择K个单位（模为1）正交基，使得原始数据变换到这组基上后，各字段两两间协方差为0，而字段的方差则尽可能大（在正交的约束下，取最大的K个方差）。